# **Task 1 - Exploratory Data Analysis**

## **Section 1 - Setup**


In order to view, analyse and manipulate the dataset, we must load it into something called a `dataframe`, which is a way of storing tabulated data in a virtual table. This dataframe will allow us to analyse the data freely. To load it into a dataframe, we will need a package called `Pandas`. We can install pandas with this command:

In [61]:
!pip install pyspark pingouin -qqq
import pandas as pd
import plotly.express as px
import seaborn as sns
import pingouin as pg
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('name').getOrCreate()

## **Section 2 - Data loading**

Now that Google Drive is mounted, you can store the CSV file anywhere in your Drive and update the `path` variable below to access it within this notebook. Once we've updated the `path`, let's read this CSV file into a pandas dataframe and see what it looks like

In [62]:
path = "sample_sales_data.csv"
df = pd.read_csv(path)
df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
spark.createDataFrame(df.head()).show()

+--------------------+-------------------+--------------------+--------+-------------+----------+--------+-----+------------+
|      transaction_id|          timestamp|          product_id|category|customer_type|unit_price|quantity|total|payment_type|
+--------------------+-------------------+--------------------+--------+-------------+----------+--------+-----+------------+
|a1c82654-c52c-45b...|2022-03-02 09:51:38|3bc6c1ea-0198-46d...|   fruit|         gold|      3.99|       2| 7.98|    e-wallet|
|931ad550-09e8-4da...|2022-03-06 10:33:59|ad81b46c-bf38-41c...|   fruit|     standard|      3.99|       1| 3.99|    e-wallet|
|ae133534-6f61-4cd...|2022-03-04 17:20:21|7c55cbd4-f306-4c0...|   fruit|      premium|      0.19|       2| 0.38|    e-wallet|
|157cebd9-aaf0-475...|2022-03-02 17:23:58|80da8348-1707-403...|   fruit|         gold|      0.19|       4| 0.76|    e-wallet|
|a81a6cd3-5e0c-44a...|2022-03-05 14:32:43|7f5e86e6-f06f-45f...|   fruit|        basic|      4.49|       2| 8.98|  debi

## **Section 3 - Descriptive statistics**

From just looking at the data, it is hard to get a feeling of what all the columns and rows mean. To gain an understanding of the dataset, let's first look at what columns and datatypes we have

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7829 entries, 0 to 7828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7829 non-null   object 
 1   timestamp       7829 non-null   object 
 2   product_id      7829 non-null   object 
 3   category        7829 non-null   object 
 4   customer_type   7829 non-null   object 
 5   unit_price      7829 non-null   float64
 6   quantity        7829 non-null   int64  
 7   total           7829 non-null   float64
 8   payment_type    7829 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 550.6+ KB


The `.info()` method creates a data description of the dataframe. It tells us:

- How many rows (7829)
- How many columns (8)
- Column names
- How many values are non-null for each column
- The types of data contained within each column
- The size of the dataset loaded into memory (~550KB)

Looking at the output of the `.info()` method, we can intepret each column as follows:

- transaction_id = this is a unique ID that is assigned to each transaction
- timestamp = this is the datetime at which the transaction was made
- product_id = this is an ID that is assigned to the product that was sold. Each product has a unique ID
- category = this is the category that the product is contained within
- customer_type = this is the type of customer that made the transaction
- unit_price = the price that 1 unit of this item sells for
- quantity = the number of units sold for this product within this transaction
- total = the total amount payable by the customer
- payment_type = the payment method used by the customer

It is also interesting to look at the datatypes. We can see that there are 3 different datatypes within this dataset:

- object = this column contains categorical values
- float64 = this column contains floating point numerical values (i.e. decimal numbers)
- int64 = this column contains integer values (whole numbers)

Now let's compute some descriptive statistics of the numeric columns:

In [66]:
df.describe()

,unit_price,quantity,total
count,7829.000000,7829.000000,7829.000000
mean,7.819480,2.501597,19.709905
std,5.388088,1.122722,17.446680
min,0.190000,1.000000,0.190000
25%,3.990000,1.000000,6.570000
50%,7.190000,3.000000,14.970000
75%,11.190000,4.000000,28.470000
max,23.990000,4.000000,95.960000


The `.describe()` method computes some descriptive statistics of the numerical columns, including:

- count = count of how many unique values exist
- mean = mean average value of this column
- std = standard deviation
- min = minimum value
- 25% = lower quartile value
- 50% = median value
- 75% = upper quartile value
- max = maximum value

## **Section 4 - Statistical Tests**


Let's do some statistical testing to determine if there is a significant association between two categorical variables, for this we will use chi2 test

In [ ]:
def crosstables(df,cat1,cat2):

    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>> Frequencies <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    print('')
    w=pd.crosstab(df[cat1],df[cat2])
    print(w)
    print('')
    print('>>>>>>>>>>>>>>>>>>>>>>>> Chi-square test <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    e,o,st=pg.chi2_independence(data=df,x=cat1,y=cat2)
    print(f"Chi-Square Statistic:")
    print(spark.createDataFrame(st.round(3)).show())

# crosstables(df,'customer_type','quantity')
# crosstables(df,'customer_type','payment_type')
# crosstables(df,'payment_type','quantity')
# crosstables(df,'category','customer_type')
crosstables(df,'category','quantity')

To analyse the dataset, below are snippets of code that you can use as helper functions to visualise different columns within the dataset. They include:

- plot_continuous_distribution = this is to visualise the distribution of numeric columns
- get_unique_values = this is to show how many unique values are present within a column
- plot_categorical_distribution = this is to visualise the distribution of categorical columns


## **Section 5 - Visualisation**

These statistics are useful, but they are better understood using visualisations. For visualisation, we will use the `seaborn` package, which makes it really easy to create visualisations from a dataframe.

### **Numerical Columns**

In [67]:
def plot_continuous_distribution(data: pd.DataFrame = None, column: str = None, height=350,width=700):
  px.histogram(data,x=column,marginal='box',height=height,width=width,template='plotly_white').show()

def get_unique_values(data, column):
  num_unique_values = len(data[column].unique())
  value_counts = data[column].value_counts()
  print(f"Column: {column} has {num_unique_values} unique values\n")
  print(value_counts)

def count_values(data: pd.DataFrame = None, column: str = None, height=300,width=700):
    counts = data.groupby(column).size().reset_index(name="counts")
    px.bar(counts,height=height,width=width,template='plotly_white').show()

In [68]:
plot_continuous_distribution(df, 'unit_price')

This tell us that the distribution of `unit_price` is **positively skewed**, that is, there are more sales of products with a low unit_price compared to products with a high unit_price.

This makes sense, you would expect a grocery store to sell more products that are cheap, and just a few products that are really expensive.

In [69]:
plot_continuous_distribution(df, 'quantity')

The distribution of `quantity` is very different. We can see that only 4 unique values exist (1, 2, 3, and 4) and they are quite evenly distributed. It seems as though customers are buying in even quantities across 1 to 4 units

In [70]:
plot_continuous_distribution(df, 'total')

The `total` follows a similar distribution to `unit_price`. This you may expect, as the total is calculated as `unit_price x quantity`.

However, this distribution is even more positively skewed. Once again, using intuition, this distribution makes sense. You'd expect customers at a grocery store to generally make more transactions of low value and only occasionally make a transaction of a very high value.

### **Categorical Columns**

Now let's turn our attention to the categorical columns within the dataset.

Before visualising these columns, it is worth us understanding how **many unique values these columns have**. If a categorical column has 1000's of unique values, it will be very difficult to visualise.


In [71]:
get_unique_values(df, 'transaction_id')

Column: transaction_id has 7829 unique values

a1c82654-c52c-45b3-8ce8-4c2a1efe63ed    1
6532e258-95fd-4eb5-8c67-2bfb879a8fec    1
6fce2af3-47a0-4755-99c9-0cefb5ab6f41    1
6476e388-3990-471f-b415-3ee59ae18832    1
10afe89b-c45b-49a2-b0be-dec89a4c3f80    1
                                       ..
a9abe5ac-99d5-4d8b-bbbd-c2a207642849    1
6b0b23e8-412b-4665-8cc4-3e37f0d9e195    1
711a4162-1985-4f5a-94ca-137cfacaeadf    1
7d1e9010-dbaf-4770-a467-f31477910f7a    1
afd70b4f-ee21-402d-8d8f-0d9e13c2bea6    1
Name: transaction_id, Length: 7829, dtype: int64


As explained previously, `transaction_id` is a unique ID column for each transaction. Since each row represents a unique transaction, this means that we have 7829 unique transaction IDs. Therefore, this column is not useful to visualise.

In [72]:
get_unique_values(df, 'product_id')

Column: product_id has 300 unique values

ecac012c-1dec-41d4-9ebd-56fb7166f6d9    114
80da8348-1707-403f-8be7-9e6deeccc883    109
0ddc2379-adba-4fb0-aa97-19fcafc738a1    108
7c55cbd4-f306-4c04-a030-628cbe7867c1    104
3bc6c1ea-0198-46de-9ffd-514ae3338713    101
                                       ... 
49f7d4a9-713a-4824-b378-aebb33ff8b2f      5
a8fab83a-16d4-4db0-a83a-f824ecd8604a      5
c8de27d0-2c44-4b5a-b178-59c45d054ccb      5
5adfc643-aa8e-4140-b2c3-98a946444632      5
ec0bb9b5-45e3-4de8-963d-e92aa91a201e      3
Name: product_id, Length: 300, dtype: int64


Similarly, `product_id` is an ID column, however it is unique **based on the product that was sold** within the transaction. From this computation, we can see that **we have 300 unique product IDs**, hence 300 unique products within the dataset. This is not worth visualising, but it certainly interesting to know.

From the output of the helper function, we can see that the product most frequently was sold within this dataset was `ecac012c-1dec-41d4-9ebd-56fb7166f6d9`, sold 114 times during the week. Whereas the product least sold was `ec0bb9b5-45e3-4de8-963d-e92aa91a201e` sold just 3 times


In [73]:
get_unique_values(df, 'category')

Column: category has 22 unique values

fruit                    998
vegetables               846
packaged foods           507
baked goods              443
canned foods             431
refrigerated items       425
kitchen                  382
meat                     382
dairy                    375
beverages                301
cheese                   293
cleaning products        292
baking                   264
snacks                   263
frozen                   263
seafood                  253
medicine                 243
baby products            224
condiments and sauces    181
personal care            177
pets                     161
spices and herbs         125
Name: category, dtype: int64


There are 22 unique values for `category`, with `fruit` and `vegetables` being the 2 most frequently purchased product categories and `spices and herbs` being the least. Let's visualise this too

In [74]:
def count_values_quantity(data: pd.DataFrame = None, column: str = None, height=300,width=700):
    counts = data.groupby(column).count()['quantity'].sort_values(ascending=False)
    px.bar(counts,height=height,width=width,template='plotly_white').show()

count_values_quantity(df,'category')

In [75]:
def count_values(data: pd.DataFrame = None, column: str = None, height=300,width=700):
    counts = data.groupby(column).count()['quantity'].sort_values(ascending=False)
    px.bar(counts,height=height,width=width,template='plotly_white').show()

count_values(df,'customer_type')

There are 5 unique values for `customer_type`, and they seem to be evenly distributed. Let's visualise this:

From this sample of data, non-members appear to be the most frequent type of customers, closely followed by standard and premium customers

In [76]:
count_values(df,'payment_type')

There are 4 unique values for `payment_type`, and they seem to be quite evenly distributed once again. Let's visualise this:

Interestingly, cash seems to be the most frequently used method of payment from this sample of data, with debit cards being the least frequent.


This dataset is a sample from 1 store across 1 week. So it will be interesting to see if the population sample follows similar patterns.

In [77]:
get_unique_values(df, 'timestamp')

Column: timestamp has 7738 unique values

2022-03-02 19:32:20    2
2022-03-05 10:11:33    2
2022-03-01 18:12:17    2
2022-03-03 13:28:14    2
2022-03-07 17:13:29    2
                      ..
2022-03-01 10:00:58    1
2022-03-02 11:05:58    1
2022-03-04 17:48:49    1
2022-03-04 14:53:27    1
2022-03-06 13:50:36    1
Name: timestamp, Length: 7738, dtype: int64


Clearly there are a lot of unique values for the timestamp column.

However, you may have noticed something...

The column named `timestamp` appears to be categorical, but in actual fact it's not. This is a datetime, following the format of `2022-03-01 10:00:45 = YYYY-MM-DD HH:MM:SS`. Therefore, we must transform this column to reflect its true form.

A helper function is provided below to convert the column into a datetime column.

In [78]:
def convert_to_datetime(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  dummy[column] = pd.to_datetime(dummy[column], format='%Y-%m-%d %H:%M:%S')
  return dummy

In [79]:
df = convert_to_datetime(df, 'timestamp')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7829 entries, 0 to 7828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  7829 non-null   object        
 1   timestamp       7829 non-null   datetime64[ns]
 2   product_id      7829 non-null   object        
 3   category        7829 non-null   object        
 4   customer_type   7829 non-null   object        
 5   unit_price      7829 non-null   float64       
 6   quantity        7829 non-null   int64         
 7   total           7829 non-null   float64       
 8   payment_type    7829 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 550.6+ KB


Using the `.info()` method again, we can see that the timestamp is now of type `datetime64[ns`, which indicates it is a datetime based data type. Now that this is a datetime column, we can explode this column out into its consitituent parts, e.g. we can explode datetime into `hour` for example.

In [81]:
df['hour'] = df['timestamp'].dt.hour

In [82]:
df.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type,hour
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:51:38,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet,9
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:33:59,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet,10
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:20:21,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet,17
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:23:58,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet,17
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:32:43,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card,14


In [83]:
get_unique_values(df, 'hour')

Column: hour has 11 unique values

11    738
16    730
18    727
10    723
12    721
9     718
19    709
14    702
17    699
13    693
15    669
Name: hour, dtype: int64


In [84]:
count_values(df,'hour')

From this we can see that the 11th, 16th and 18th hour of the day are the top 3 hours of the day for transactions being processed. This is interesting, this would suggest that their busiest times of day may be just before lunch, and as people are on the way home from work. Once again, this is a small sample of data, so we can't make assumptions on the population sample of data, but it gives us insights to go back to the business with.

## **Section 6 - Correlations**

By now, you should have a good understanding of all the columns within the dataset, as well as the values that occur within each column. One more thing that we can do is to look at how each of the numerical columns are related to each other.

To do this, we can use `correlations`. Correlations measure how each numeric column is linearly related to each other. It is measured between -1 and 1. If a correlation between 2 columns is close to -1, it shows that there is a negative correlation, that is, as 1 increases, the other decreases. If a correlation between 2 columns is close to 1, it shows that they are positively correlated, that is, as 1 increases, so does the other. Therefore, correlations do not infer that one column causes the other, but it gives us an indication as to how the columns are linearly related.

In [85]:
corr = df.corr()
corr

<ipython-input-85-4381f08f6434>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,unit_price,quantity,total,hour
unit_price,1.000000,0.024588,0.792018,0.010842
quantity,0.024588,1.000000,0.521926,-0.008410
total,0.792018,0.521926,1.000000,0.010354
hour,0.010842,-0.008410,0.010354,1.000000


From this correlation matrix, we can see that the only columns that have a high correlation are `unit_price` and `total`. This is understandable because total is calculated used unit_price.

All the other correlations are close to 0, indicating that there is not a significant positive or negative correlation between the numeric variables.

## **Section 7 - Summary**

We have completed an initial exploratory data analysis on the sample of data provided. We should now have a solid understanding of the data.

**The client wants to know**

```
"How to better stock the items that they sell"
```

From this dataset, it is impossible to answer that question. In order to make the next step on this project with the client, it is clear that:

- We need more rows of data. The current sample is only from 1 store and 1 week worth of data
- We need to **frame the specific problem statement** that we want to solve. The current business problem is too broad, we should narrow down the focus in order to deliver a valuable end product
- We **need more features**. Based on the problem statement that we move forward with, we need more columns (features) that may help us to understand the outcome that we're solving for

# **Task 1 - Client Request Analysis**

The client wants to know

```
"How to better stock the items that they sell"
```

From this dataset, it is impossible to answer that question. In order to make the next step on this project with the client, it is clear that:

- We need more rows of data. The current sample is only from 1 store and 1 week worth of data
- We need to frame the specific problem statement that we want to solve. The current business problem is too broad, we should narrow down the focus in order to deliver a valuable end product
- We need more features. Based on the problem statement that we move forward with, we need more columns (features) that may help us to understand the outcome that we're solving for

## **Initial Thoughts**

Its evident that the request is mostly associated with timings of stocking logistics. We ought to know which products should be stocked and at which intervals, based on the user/store demand, so its likely this task will be formulated as a **time series** regression problem. Lets assume that the business restocks every week and have optimally configured their demand/stock for this store.

## **More Data Insights**

Let's do some more data insights, so we understand our data a little bit better


In [86]:
df['dow'] = df['timestamp'].dt.dayofweek
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['date'] = df['timestamp'].apply(lambda x: x.date()).astype('str')
df.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type,hour,dow,year,month,date
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:51:38,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet,9,2,2022,3,2022-03-02
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:33:59,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet,10,6,2022,3,2022-03-06
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:20:21,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet,17,4,2022,3,2022-03-04
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:23:58,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet,17,2,2022,3,2022-03-02
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:32:43,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card,14,5,2022,3,2022-03-05


In [112]:
px.box(df.sort_values('unit_price'),x='category',y='unit_price',template='plotly_white',title='<b>unit_price</b> statistics across <b>categories</b>')

In [126]:
px.box(df.sort_values(by='total'),x='category',y='total',template='plotly_white',title='<b>total</b> statistics across <b>categories</b>',height=400)

### **Variations amongst customer types**

Let's look at the mean statistics for different customer types

In [175]:
# mean value across category & customer type
cat_cust_mean = df.sort_values('unit_price').groupby(['category','customer_type'],as_index=False).mean()
cat_cust_sum = df.sort_values('unit_price').groupby(['category','customer_type','payment_type'],as_index=False).sum()
cat_cust_mean.head()

<ipython-input-175-ff1a9b3534f2>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-175-ff1a9b3534f2>:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,category,customer_type,unit_price,quantity,total,hour,dow,year,month
0,baby products,basic,12.241020,2.612245,31.543265,13.346939,2.795918,2022.0,3.0
1,baby products,gold,12.040000,2.326087,28.481087,14.565217,2.586957,2022.0,3.0
2,baby products,non-member,12.407778,2.266667,28.741778,13.911111,2.666667,2022.0,3.0
3,baby products,premium,12.746410,2.615385,33.771282,14.564103,3.076923,2022.0,3.0
4,baby products,standard,12.063333,2.311111,27.528000,14.155556,3.266667,2022.0,3.0


In [129]:
px.bar(cat_cust_mean,x='category',y='total',color='customer_type',template='plotly_white',title='total procurement mean value across <b>customer_type</b>',barmode='group')

In [130]:
px.line(cat_cust_mean,x='category',y='quantity',color='customer_type',template='plotly_white',title='total procurement count across <b>customer_type</b>')

In [176]:
fig = px.bar(cat_cust_sum.sort_values(by=['total']),y='category',x='total',template='plotly_white',facet_col='payment_type',facet_col_wrap=2,
             title='total procurement sum across <b>customer_type</b> and <b>category</b>',text_auto=True,height=1000,)
fig.update_layout(yaxis={'categoryorder':'total ascending'}) # sort by total

### **Variations amongst customer types (by date)**

This time lets look at their spending amount & item count **grouped by date**

In [91]:
# mean value across category & customer type
cat_cust_mean = df.sort_values('unit_price').groupby(['customer_type','date'],as_index=False).mean()
cat_cust_mean.head()

<ipython-input-91-ddbd72649d04>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,customer_type,date,unit_price,quantity,total,hour,dow,year,month
0,basic,2022-03-01,7.700044,2.441048,18.660306,13.934498,1.0,2022.0,3.0
1,basic,2022-03-02,7.393448,2.551724,18.531810,14.047414,2.0,2022.0,3.0
2,basic,2022-03-03,7.593000,2.415000,18.560350,13.820000,3.0,2022.0,3.0
3,basic,2022-03-04,7.616667,2.466667,19.426889,14.124444,4.0,2022.0,3.0
4,basic,2022-03-05,8.182070,2.475771,20.254978,14.211454,5.0,2022.0,3.0


In [92]:
fig = px.bar(cat_cust_mean.sort_values(['total']),x='total',template='plotly_white',title='total procurement mean value across <b>customer_type</b> and <b>date</b>',y='customer_type',color='date',text_auto=True)
# fig.update_layout(yaxis=dict(autorange="reversed"))
fig.update_layout(yaxis={'categoryorder':'total ascending'}) # sort by total

### **Cash Spending Insight**

We found cash is most popular among purchases, lets explore a little more about this

In [93]:
payment_type_diff = df.groupby(['payment_type','customer_type','category'],as_index=False).count()[['payment_type','customer_type','category','quantity']]
payment_type_diff.head(4)

,payment_type,customer_type,category,quantity
0,cash,basic,baby products,9
1,cash,basic,baked goods,26
2,cash,basic,baking,18
3,cash,basic,beverages,22
